In [1]:
import sys
sys.path.append('../')
import nbimporter
from basic.SaturatedVaporPressure import get_saturated_vapor_pressure as get_svp
from basic.VaporPressure import rh_to_ah, ah_to_rh, ah_to_pv, pv_to_ah, pv_to_rh, get_saturated_absolute_humidity, get_dew_point_temp
from library.Temperature import Temperature

import matplotlib.pyplot as plt

Importing Jupyter notebook from ..\basic\SaturatedVaporPressure.ipynb
Importing Jupyter notebook from ..\basic\VaporPressure.ipynb
Importing Jupyter notebook from ..\library\Temperature.ipynb


# Air Class

## 1. Class

In [2]:
class Air:
    
    svpeq = 'SONNTAG' # the default equation is based no SONNTAG
    eqtype = 'water'
    
    def __init__(self, t, h, humtype):
        self._T = t
        self.set_humidity(t, h, humtype)
    
    def set_humidity(self, t, h, humtype):
        f = {
            'relative' : self.set_relative_humidity,
            'absolute' : self.set_absolute_humidity
        }[humtype]
        f(h)
    
    # temperature
    
    def set_temperature(self,t):
        if ah_to_rh(self._AH, t.K, self.svpeq, self.eqtype) > 100:
            raise ValueError('Error: Temperature shall be not set at the state that the relative humidity is over 100 %.')
        self._T = t

    def get_temperature(self):
        return self._T
    
    T = property(get_temperature, set_temperature)

    # relative humidity
    
    def set_relative_humidity(self, rh):
        if rh < 0:
            raise ValueError('Error: Relative humidity shall be equal to or more than 0 %.')
        if rh > 100:
            raise ValueError('Error: Relative humidity shall not be over 100 %.')
        self._AH = rh_to_ah(rh, self._T.K, self.svpeq, self.eqtype)
    
    def get_relative_humidity(self):
        return ah_to_rh(self._AH, self._T.K, self.svpeq, self.eqtype)
    
    RH = property(get_relative_humidity, set_relative_humidity)
    
    # absolute humidity
    
    def set_absolute_humidity(self, ah):
        if ah < 0:
            raise ValueError('Error: Absolute humidity shall be equal to or more than 0 kg/kgDA.')
        if ah_to_rh(ah, self._T.K, self.svpeq, self.eqtype) > 100:
            raise ValueError('Error: Absolute humidity shall be not set at the state that the relative humidity is over 100 %.')
        self._AH = ah
        
    def get_absolute_humidity(self):
        return self._AH
    
    AH = property(get_absolute_humidity, set_absolute_humidity)
    
    # vapor pressure
    
    def set_vapor_pressure(self, pv):
        if pv < 0:
            raise ValueError('Error: Vapor pressure shall be equal to or more than 0 Pa.')
        if pv_to_rh(pv, self._T.K, self.svpeq, self.eqtype) > 100:
            raise ValueError('Error: Vapor pressure shall be not set at the state that the relative humidity is over 100 %.')
        self._AH = pv_to_ah(pv)
    
    def get_vapor_pressure(self):
        return ah_to_pv(self._AH)
    
    VP = property(get_vapor_pressure, set_vapor_pressure)
    
    # saturated vapor pressure
    
    def get_SVP(self):
        return get_svp(self.svpeq, self.eqtype, self._T.K)
    
    SVP = property(get_SVP, None)
    
    # saturated absolute humidity

    def get_SAH(self):
        return get_saturated_absolute_humidity(self._T.K, self.svpeq, self.eqtype)
    
    SAH = property(get_SAH, None)
    
    # dew point temperature
    
    def get_DPT(self):
        return Temperature(get_dew_point_temp(self._AH, self.svpeq, self.eqtype))
    
    DPT = property(get_DPT, None)

## 2. Mannual

### 2.1 Set

At first, set the parameters of constructor.  
The first parameter is temperature as 'Temperature' class.  
The second parameter is humidity. Which this value means relative (%) or absolute (kg/kgDA) is depending on the therd parameter.  
The therd parameter represents the second parameter's meaning. You can specify 'relative' or 'absolute'.

The below is an example.

In [3]:
a = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

Relative humidity shall be 0 ~ 100 (%).

In [4]:
try:
    Air( Temperature(35.0,'Celsius'), -1.0, 'relative' )
except ValueError as e:
    print(e)

Error: Relative humidity shall be equal to or more than 0 %.


In [5]:
try:
    Air( Temperature(35.0,'Celsius'), 101.0, 'relative' )
except ValueError as e:
    print(e)

Error: Relative humidity shall not be over 100 %.


Absolute humidity shall not be over the saturated absolute humidity. 

In [6]:
try:
    Air( Temperature(35.0,'Celsius'), 0.037, 'absolute' )
        # saturated absolute humidity at 35 degree C is 0.036588480692732184 based on SONNTAG equation.
except ValueError as e:
    print(e)

Error: Absolute humidity shall be not set at the state that the relative humidity is over 100 %.


## 2.2 Get

You can get the parameters by using property.

### 1) Temperature

In [7]:
a.T, a.T.C, a.T.K  # class, degree C, K 

(<library.Temperature.Temperature at 0x175f3ccab00>, 35.0, 308.15)

### 2) Relative Humidity

In [8]:
a.RH # %

50.0

### 3) Absolute Humidity

In [9]:
a.AH # kg/kgDA

0.017771544359487364

### 4) Vapor Pressure

In [10]:
a.VP # Pa

2814.601162088015

### 5) Saturated Vapor Pressure

In [11]:
a.SVP # Pa

5629.20232417603

### 6) Saturated Absolute Humidity

In [12]:
a.SAH # kg/kgDA

0.036588480692732184

### 7) Dew Point Temperature

In [13]:
a.DPT, a.DPT.K, a.DPT.C

(<library.Temperature.Temperature at 0x175f3d0dc50>,
 296.1710157394409,
 23.02101573944094)

### 2.3 Change Parameter

You can change parameters.

### 1) Temperature

In the example below, the temperature is changed to 40.0 degree C from 35.0 degree C. 

In [14]:
a2 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [15]:
a2.T.C, a2.RH, a2.AH

(35.0, 50.0, 0.017771544359487364)

In [16]:
a2.T = Temperature(40.0,'Celsius')

In [17]:
a2.T.C, a2.RH, a2.AH

(40.0, 38.110878995581295, 0.017771544359487364)

You can not set the temperature which is below the dew point temperature.

In [18]:
a3 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [19]:
a3.DPT.C

23.02101573944094

In [20]:
try:
    a3.T = Temperature(23.0,'Celsius') # The dew point temperature is 23.02101573944094 degree C.
except ValueError as e:
    print(e)

Error: Temperature shall be not set at the state that the relative humidity is over 100 %.


### 2) Absolute Humidity

In the example below, the absolute humidity is changed to 0.019 kg/kgDA from 0.017 kg/kgDA. 

In [21]:
a4 = Air( Temperature(35.0,'Celsius'), 0.017, 'absolute' )

In [22]:
a4.T.C, a4.RH, a4.AH

(35.0, 47.88702060713158, 0.017)

In [23]:
a4.AH = 0.019

In [24]:
a4.T.C, a4.RH, a4.AH

(35.0, 53.353796199980216, 0.019)

You can not set the absolute humidity which is over the saturated absolute humidity.

In [25]:
a5 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [26]:
a5.SAH

0.036588480692732184

In [27]:
try:
    a5.AH = 0.0366 # The saturated absolute humidity is 0.036588480692732184(kg/kgDA).
except ValueError as e:
    print(e)

Error: Absolute humidity shall be not set at the state that the relative humidity is over 100 %.


### 3) Relative Humidity

In the example below, the relative humidity is changed to 75.0 % from 50.0 %. 

In [28]:
a6 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [29]:
a6.T.C, a6.RH, a6.AH

(35.0, 50.0, 0.017771544359487364)

In [30]:
a6.RH = 75.0

In [31]:
a6.RH

75.0

You can not set the relative humidity which is out of range of 0 ~ 100 %.

In [32]:
a7 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [33]:
try:
    a7.RH = -1.0
except ValueError as e:
    print(e)

Error: Relative humidity shall be equal to or more than 0 %.


In [34]:
try:
    a7.RH = 101.0
except ValueError as e:
    print(e)

Error: Relative humidity shall not be over 100 %.


### 4) Vapor Pressure

In the example below, the vapor pressure is changed to 3000.0 Pa from 2814.601162088015 Pa. 

In [35]:
a8 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [36]:
a8.VP

2814.601162088015

In [37]:
a8.VP = 3000.0

In [38]:
a8.VP

3000.0

You can not set the vapor pressure which is over the saturated vapor pressure.

In [39]:
a9 = Air( Temperature(35.0,'Celsius'), 50.0, 'relative' )

In [40]:
a9.SVP

5629.20232417603

In [41]:
try:
    a9.VP = 5630.0 # The saturated vapor pressure is 5629.20232417603(Pa).
except ValueError as e:
    print(e)

Error: Vapor pressure shall be not set at the state that the relative humidity is over 100 %.
